In [16]:
import os
import pandas as pd
import re
import glob
import numpy as np


In [17]:
# === CONFIGURE YOUR WALL BOUNDS HERE ===
xmin = -5.5
xmax = 5.5
zmin = -16.5
zmax = 4.5



In [18]:
# === DIRECTORY OF PLAYER CSV FILES ===
data_folder = "Data/Unity Data/All"  # Replace this with your folder path

# === OUTPUT ===
output_rows = []

In [19]:
id=5
df=pd.read_csv(glob.glob(f'Data/Unity Data/All/{id}_*.csv')[0])

In [20]:
# Helper function to parse EventPosition field
def parse_position(pos_str):
    try:
        x, _, z = map(float, pos_str.split())
        return x, z
    except:
        return None, None

for file in os.listdir(data_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(data_folder, file)
        df = pd.read_csv(file_path, on_bad_lines='skip')

        player_id = os.path.splitext(file)[0].split("_")[0]
        total_near_wall_time = 0

        earthquake_active = False
        last_timestamp = None

        # Clean up column names (in case they have extra spaces)
        df.columns = [col.strip() for col in df.columns]

        for i, row in df.iterrows():
            event_type = str(row.get("EventType", "")).strip()
            timestamp = row.get("Time", None)

            if pd.isna(timestamp):
                continue

            if event_type == "EarthquakeStart":
                earthquake_active = True
            elif event_type == "EarthquakeEnd":
                earthquake_active = False

            if event_type == "RealtimeData" and earthquake_active:
                pos_str = str(row.get("EventPosition(X Y Z)", ""))
                x, z = parse_position(pos_str)

                if x is not None and z is not None:
                    is_near_wall = (x <= xmin or x >= xmax or z <= zmin or z >= zmax)

                    if is_near_wall:
                        if last_timestamp is not None:
                            dt = (int(timestamp) - int(last_timestamp)) / 1000.0  # convert ms to seconds
                            total_near_wall_time += dt
                        last_timestamp = int(timestamp)
                    else:
                        last_timestamp = int(timestamp)
                else:
                    last_timestamp = None
            else:
                last_timestamp = None

        output_rows.append({
            "PlayerID": player_id,
            "TimeNearWallDuringEarthquake(s)": round(total_near_wall_time, 2)
        })

# Save result
output_df = pd.DataFrame(output_rows)
output_df.to_csv("near_wall_time_summary.csv", index=False)

print("✅ Analysis complete. Results saved to 'near_wall_time_summary.csv'.")


✅ Analysis complete. Results saved to 'near_wall_time_summary.csv'.


In [ ]:
# Read the near_wall_time_summary.csv file
summary_df = pd.read_csv("near_wall_time_summary.csv")
group_mapping = pd.read_csv("Data/Unity Data/All/Group.csv")  # Adjust the path as necessary

# Extract the 'Group' column from the original dataframe and merge it with the summary dataframe
# group_mapping = df[['ID', 'Group']].dropna().drop_duplicates()
group_mapping['ID'] = group_mapping['ID'].astype(str)  # Ensure IDs are strings for matching
summary_df['Group'] = summary_df['PlayerID'].str.split('_').str[1].map(group_mapping.set_index('ID')['Group'])

# Save the updated summary dataframe back to a CSV file
summary_df.to_csv("near_wall_time_summary_with_group.csv", index=False)

print("✅ Group column added and saved to 'near_wall_time_summary_with_group.csv'.")

AttributeError: Can only use .str accessor with string values!